In [ ]:
# XGBoost classification - halvinggridsearch

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder

data = pd.read_json('./content/video_games_newer_all_data_with_rawg_final_for_train.json')

data['User_Score'] = np.where(data['User_Score'] <=3.5, 0, data['User_Score'])
data['User_Score'] = np.where((3.5 < data['User_Score']) & ( data['User_Score'] <=7.5), 1 ,data['User_Score'])
data['User_Score'] = np.where(7.5 < data['User_Score'] , 2 ,data['User_Score'])
# data['User_Score'] = data['User_Score'].astype('category').cat.codes

le = LabelEncoder()

training, test = train_test_split(data, test_size = 0.25, random_state = 32)

features = training.drop(['User_Score'], axis = 1).columns

training_features, training_labels = training[features], le.fit_transform(training['User_Score'])
test_features, test_labels = test[features], le.fit_transform(test['User_Score'])

params = { 
    'classifier__max_depth': [3,6,10],
    'classifier__learning_rate': [0.001, 0.005, 0.01],
    'classifier__n_estimators': [100, 500, 1000],
    'classifier__colsample_bytree': [0.1, 0.3, 0.7],
    'classifier__lambda': [1],
    'classifier__gamma': [0]
}

xgb_pipeline = Pipeline([('scaler', StandardScaler()), ('classifier', XGBClassifier())])
clf = HalvingGridSearchCV(
    estimator=xgb_pipeline, 
    param_grid=params,               
    scoring='f1_macro', 
    verbose=1
)
clf.fit(training_features, training_labels)
print("Best parameters:", clf.best_params_)
print("Best f1-macro: ", clf.best_score_)
f1_macro = clf.score(test_features, test_labels)
print(f'Score {f1_macro}')

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 35
max_resources_: 2851
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 81
n_resources: 35
Fitting 5 folds for each of 81 candidates, totalling 405 fits
